# TUTORIAL FOR DATA PROCESSING

In [1]:
import numpy as np
from pathlib import Path
import os
import shutil
import pandas as pd
import torch
from torch import optim
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader, RandomSampler, WeightedRandomSampler
from torchvision import models, transforms
from PIL import Image
from tqdm.auto import tqdm

import cv2
import cProfile as profile

from nvidia.dali.pipeline import Pipeline
import nvidia.dali.fn as fn
import nvidia.dali.types as types
from nvidia.dali.plugin.pytorch import DALIClassificationIterator as PyTorchIterator
from nvidia.dali.plugin.pytorch import DALIGenericIterator as GenIterator
from nvidia.dali.plugin.pytorch import LastBatchPolicy
import math

DEVICE = 'cuda:1'
DEVICE = DEVICE if torch.cuda.is_available() else 'cpu'
THREADS = 1
LOGSDIR = './runs'
# DATADIR = Path('./data')
IMAGE_DATA = '../imagenet/ILSVRC/Data/CLS-LOC/'
EPOCHS = 300
BATCHSIZE = 64
# MU = 7
TOTAL_ITERS = None
NUMWORKERS = 4

In [2]:
from utils.logger import AutoLogger
from sklearn.metrics import precision_score

In [3]:
from random import shuffle

## Image Data Loading

In [4]:
# Image data
DATADIR = Path('../semisupervisedMM/data')
IMAGE_DATA = '../imagenet/ILSVRC/Data/CLS-LOC/'
datapath = Path(IMAGE_DATA)
imagepath = datapath/'train'
validpath = datapath/'val'

In [5]:
labelled = pd.read_csv(DATADIR/'labelled.csv')
test = pd.read_csv(DATADIR/'test.csv')

In [6]:
class SupervisedDataset(Dataset):
    def __init__(self, path, df, weak_trans, 
            batch_size, label_list=None, total_len=0, shuffle=False):
        self.df = df
        # self.labelled = is_labelled
        self.total_len = total_len
        # self.strong_trans = strong_trans
        self.weak_trans = weak_trans
        self.path = Path(path)
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.label_list = label_list
        # if self.labelled:
        if not self.label_list:
            self.label_list = self.df.label.unique().tolist()
        class_w = (self.df.shape[0]/self.df.label.value_counts())
        # print(class_w)

        self.weights = self.df.label.apply(lambda x: class_w[x]).values

    def __len__(self):
        return len(self.df)

    def __iter__(self):
        self.indices = list(range(len(self)))
        if self.shuffle:
            shuffle(self.indices)
        self.total_batches = 0
        self.i = 0
        self.n = len(self)
        return self
    
    def __next__(self):
        batch = []
        batch_2 = []

        for _ in range(self.batch_size):
            if (self.i == self.n) and (not self.total_len):
                self.__iter__()
                raise StopIteration()
            
            
            row = self.df.iloc[self.indices[self.i]]
            image = row.path
            
            f = open(self.path/image, 'rb')
            img = (np.frombuffer(f.read(), dtype=np.uint8))    
            batch.append(img)
            f.close()

            batch_2.append(np.array([self.label_list.index(row.label)]))
            self.i = (self.i+1)
            self.total_batches += 1
            if (self.i == self.n) and (not self.total_len):
                self.__iter__()
                raise StopIteration()
            elif (self.total_len) and (self.total_batches == self.total_len):
                self.__iter__()
                raise StopIteration()
            self.i = self.i%self.n
        
        return (batch, batch_2)
            

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        image = row.path
        
        image = Image.open(self.path/image).convert('RGB')
        # image = cv2.cvtColor(cv2.imread(str(self.path/image)), cv2.COLOR_BGR2RGB)
        

        # if self.labelled:
        X = self.weak_trans(image)
        image.close()
        y = self.label_list.index(row.label)
        y = torch.tensor(y, dtype=torch.int64)
        return X, y



weak_trans = transforms.Compose([
    transforms.RandomRotation(5),
    transforms.RandomPerspective(0.05),
    transforms.RandomResizedCrop(224, (0.8, 1)),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((224,224)),
    transforms.ColorJitter(0.1, 0.1, 0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

test_trans = transforms.Compose([
    transforms.ToTensor(),
    transforms.CenterCrop((224,224)),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

In [7]:
def dali_weak_aug(image):
    image = fn.rotate(image, angle=fn.random.uniform(range=(0,5)))
    image = fn.random_resized_crop(image, size=(224,224), 
                                   random_area=(0.8,1))
    image = fn.flip(image, horizontal=fn.random.coin_flip())
    image = fn.color_twist(image, brightness=fn.random.uniform(range=(0.8,1.2)),
                        contrast=fn.random.uniform(range=(0.8,1.2)),
                        saturation=fn.random.uniform(range=(0.9,1)))
    image = fn.crop_mirror_normalize(image, 
                                mean=[123.6750, 116.2800, 103.5300], 
                                std=[58.3950, 57.1200, 57.3750],
                                dtype=types.FLOAT)
    return image


def ExternalLabelledSourcePipeline(batch_size, num_threads, device_id, external_data):
    pipe = Pipeline(batch_size, num_threads, device_id)
    with pipe:
        jpegs, labels = fn.external_source(source=external_data, num_outputs=2, 
                                           dtype=[types.UINT8, types.INT64])
        images = fn.decoders.image(jpegs, device="mixed")
        images = dali_weak_aug(images)
        pipe.set_outputs(images, labels)
    return pipe

In [8]:
labelled_iterator = SupervisedDataset(imagepath, df=labelled, 
                                        weak_trans=weak_trans, batch_size=BATCHSIZE, 
                                        total_len=None, shuffle=True)

pytorch_dl = DataLoader(labelled_iterator, batch_size=BATCHSIZE, shuffle=True, drop_last=True)

pipe_labelled = ExternalLabelledSourcePipeline(batch_size=BATCHSIZE, num_threads=THREADS, device_id = int(DEVICE.split(':')[-1]),
                              external_data = labelled_iterator)

labelled_loader = GenIterator(pipe_labelled, output_map=['x', 'y'], 
                      last_batch_padded=True, last_batch_policy=LastBatchPolicy.PARTIAL)


In [9]:
profile.run("next(iter(pytorch_dl))", sort='tottime')

         82483 function calls (82073 primitive calls) in 0.711 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      132    0.180    0.001    0.180    0.001 {method 'decode' of 'ImagingDecoder' objects}
       98    0.161    0.002    0.161    0.002 {method 'transform2' of 'ImagingCore' objects}
       10    0.083    0.008    0.083    0.008 {built-in method marshal.loads}
       64    0.064    0.001    0.064    0.001 {method 'resize' of 'ImagingCore' objects}
      192    0.050    0.000    0.050    0.000 {built-in method PIL._imaging.blend}
      226    0.014    0.000    0.014    0.000 {built-in method PIL._imaging.fill}
       64    0.012    0.000    0.012    0.000 {method 'contiguous' of 'torch._C._TensorBase' objects}
       64    0.008    0.000    0.008    0.000 {method 'div' of 'torch._C._TensorBase' objects}
      258    0.008    0.000    0.008    0.000 {method 'convert' of 'ImagingCore' objects}
      126    0.007    

In [10]:
profile.run("next(iter(labelled_loader))", sort='tottime')

         7 function calls in 0.000 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {built-in method builtins.iter}
        1    0.000    0.000    0.000    0.000 base_iterator.py:450(__iter__)
        1    0.000    0.000    0.000    0.000 pytorch.py:203(__next__)
        1    0.000    0.000    0.000    0.000 {built-in method builtins.next}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [11]:
%%timeit -n 5
next(iter(pytorch_dl))

648 ms ± 47.4 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [12]:
%%timeit -n 5
next(iter(labelled_loader))

13.2 ms ± 1.15 ms per loop (mean ± std. dev. of 7 runs, 5 loops each)


In [13]:
for batch in tqdm(labelled_loader, total=len(pytorch_dl)):
    continue

labelled_loader.reset()


  0%|          | 0/199 [00:00<?, ?it/s]

In [14]:
for batch in tqdm(pytorch_dl, total=len(pytorch_dl)):
    continue


  0%|          | 0/199 [00:00<?, ?it/s]

## Training - DALI

### With the Profiler

In [15]:
model = models.resnet18()
model.fc = nn.Linear(model.fc.in_features, len(labelled_iterator.label_list), True)
model = model.to(DEVICE)

"""
logger = AutoLogger({'precision': lambda gt, pred: (gt.argmax(dim=-1)==pred).mean()}, tensorboard=True)
for epoch in range(2):
    logger.epoch_start()
    # train_part
    for i in range(2):
        y_true = np.random.randint(0,2, size=(5,))
        y_pred = np.random.randint(0,2, size=(5,))
        logger.log_loss(0, 'train')
        logger.log_metrics(y_true, y_pred, 'train')

    # test part
    for i in range(1):
        y_true = np.random.randint(0,2, size=(5,))
        y_pred = np.random.randint(0,2, size=(5,))
        logger.log_loss(1, 'test')
        logger.log_metrics(y_true, y_pred, 'test')

    logger.epoch_end()
"""
# model = torch.nn.DataParallel(ImModel(models.resnet18, 1000, None), device_ids=[0,1,2,3]).cuda()

optimizer = optim.AdamW(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=3e-3, 
                                          epochs=EPOCHS, 
                                          steps_per_epoch=len(pytorch_dl))


with torch.profiler.profile(
        activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA],
        schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=2),
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./profiler/image-dali'),
        record_shapes=True,
        profile_memory=True,
        with_stack=True
) as prof:
    for epoch in tqdm(range(1), leave=True):
        
        epoch_train_loss = 0
        epoch_test_loss = 0
        
        for i, (labelled_data) in enumerate(tqdm(labelled_loader, total=len(pytorch_dl), leave=False)):
            # lab, unlab = batch
            inp, y = labelled_data[0]['x'], labelled_data[0]['y']
            inp = inp.to(DEVICE)
            y = y.reshape(-1).to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inp)

            loss = F.cross_entropy(outputs, y)
            

            loss.backward()
            epoch_train_loss += loss.detach().cpu()

            optimizer.step()
            scheduler.step()
            prof.step()
        
        # epoch_train_loss = epoch_train_loss/len(unlabelled_dl)
        # writer.add_scalar('Loss_Epoch', epoch_train_loss, epoch)
        labelled_loader.reset()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/199 [00:00<?, ?it/s]

STAGE:2023-08-14 10:34:59 3585955:3585955 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
[W CPUAllocator.cpp:235] Memory block of unknown size was allocated before the profiling started, profiler results will not include the deallocation event
STAGE:2023-08-14 10:34:59 3585955:3585955 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-08-14 10:34:59 3585955:3585955 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-08-14 10:35:00 3585955:3585955 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-08-14 10:35:00 3585955:3585955 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-08-14 10:35:00 3585955:3585955 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


### Without the Profiler

In [16]:
model = models.resnet18()
model.fc = nn.Linear(model.fc.in_features, len(labelled_iterator.label_list), True)
model = model.to(DEVICE)

"""
logger = AutoLogger({'precision': lambda gt, pred: (ft.argmax(dim=)).mean()}, tensorboard=True)
for epoch in range(2):
    logger.epoch_start()
    # train_part
    for i in range(2):
        y_true = np.random.randint(0,2, size=(5,))
        y_pred = np.random.randint(0,2, size=(5,))
        logger.log_loss(0, 'train')
        logger.log_metrics(y_true, y_pred, 'train')

    # test part
    for i in range(1):
        y_true = np.random.randint(0,2, size=(5,))
        y_pred = np.random.randint(0,2, size=(5,))
        logger.log_loss(1, 'test')
        logger.log_metrics(y_true, y_pred, 'test')

    logger.epoch_end()
"""
# model = torch.nn.DataParallel(ImModel(models.resnet18, 1000, None), device_ids=[0,1,2,3]).cuda()

optimizer = optim.AdamW(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=3e-3, 
                                          epochs=EPOCHS, 
                                          steps_per_epoch=len(pytorch_dl))


for epoch in tqdm(range(1), leave=True):
    
    epoch_train_loss = 0
    epoch_test_loss = 0
    
    for i, (labelled_data) in enumerate(tqdm(labelled_loader, total=len(pytorch_dl), leave=False)):
        # lab, unlab = batch
        inp, y = labelled_data[0]['x'], labelled_data[0]['y']
        inp = inp.to(DEVICE)
        y = y.reshape(-1).to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inp)

        loss = F.cross_entropy(outputs, y)
        

        loss.backward()
        epoch_train_loss += loss.detach().cpu()

        optimizer.step()
        scheduler.step()
    
    # epoch_train_loss = epoch_train_loss/len(unlabelled_dl)
    # writer.add_scalar('Loss_Epoch', epoch_train_loss, epoch)
    labelled_loader.reset()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/199 [00:00<?, ?it/s]

## Training - Native Pytorch

### With the Profiler

In [17]:
model = models.resnet18()
model.fc = nn.Linear(model.fc.in_features, len(labelled_iterator.label_list), True)
model = model.to(DEVICE)

"""
logger = AutoLogger({'precision': precision_score}, tensorboard=True)
for epoch in range(2):
    logger.epoch_start()
    # train_part
    for i in range(2):
        y_true = np.random.randint(0,2, size=(5,))
        y_pred = np.random.randint(0,2, size=(5,))
        logger.log_loss(0, 'train')
        logger.log_metrics(y_true, y_pred, 'train')

    # test part
    for i in range(1):
        y_true = np.random.randint(0,2, size=(5,))
        y_pred = np.random.randint(0,2, size=(5,))
        logger.log_loss(1, 'test')
        logger.log_metrics(y_true, y_pred, 'test')

    logger.epoch_end()
"""
# model = torch.nn.DataParallel(ImModel(models.resnet18, 1000, None), device_ids=[0,1,2,3]).cuda()

optimizer = optim.AdamW(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=3e-3, 
                                          epochs=EPOCHS, 
                                          steps_per_epoch=len(pytorch_dl))


with torch.profiler.profile(
        activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA],
        schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=2),
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./profiler/image-pytorch'),
        record_shapes=True,
        profile_memory=True,
        with_stack=True
) as prof:
    for epoch in tqdm(range(1), leave=True):
        
        epoch_train_loss = 0
        # epoch_test_loss = 0
        
        for i, (labelled_data) in enumerate(tqdm(pytorch_dl, total=len(pytorch_dl), leave=False)):
            # lab, unlab = batch
            # inp, y = labelled_data[0]['x'], labelled_data[0]['y']
            inp, y = labelled_data
            inp = inp.to(DEVICE)
            y = y.reshape(-1).to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inp)

            loss = F.cross_entropy(outputs, y)
            

            loss.backward()
            epoch_train_loss += loss.detach().cpu()

            optimizer.step()
            scheduler.step()
            prof.step()
        
        # epoch_train_loss = epoch_train_loss/len(unlabelled_dl)
        # writer.add_scalar('Loss_Epoch', epoch_train_loss, epoch)
        # labelled_loader.reset()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/199 [00:00<?, ?it/s]

STAGE:2023-08-14 10:35:16 3585955:3585955 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-08-14 10:35:19 3585955:3585955 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-08-14 10:35:19 3585955:3585955 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-08-14 10:35:25 3585955:3585955 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-08-14 10:35:27 3585955:3585955 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-08-14 10:35:27 3585955:3585955 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


### Without the Profiler

In [18]:
model = models.resnet18()
model.fc = nn.Linear(model.fc.in_features, len(labelled_iterator.label_list), True)
model = model.to(DEVICE)

"""
logger = AutoLogger({'precision': lambda gt, pred: (ft.argmax(dim=)).mean()}, tensorboard=True)
for epoch in range(2):
    logger.epoch_start()
    # train_part
    for i in range(2):
        y_true = np.random.randint(0,2, size=(5,))
        y_pred = np.random.randint(0,2, size=(5,))
        logger.log_loss(0, 'train')
        logger.log_metrics(y_true, y_pred, 'train')

    # test part
    for i in range(1):
        y_true = np.random.randint(0,2, size=(5,))
        y_pred = np.random.randint(0,2, size=(5,))
        logger.log_loss(1, 'test')
        logger.log_metrics(y_true, y_pred, 'test')

    logger.epoch_end()
"""
# model = torch.nn.DataParallel(ImModel(models.resnet18, 1000, None), device_ids=[0,1,2,3]).cuda()

optimizer = optim.AdamW(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=3e-3, 
                                          epochs=EPOCHS, 
                                          steps_per_epoch=len(pytorch_dl))


for epoch in tqdm(range(1), leave=True):
    
    epoch_train_loss = 0
    epoch_test_loss = 0
    
    for i, (labelled_data) in enumerate(tqdm(pytorch_dl, total=len(pytorch_dl), leave=False)):
        
        inp, y = labelled_data
        inp = inp.to(DEVICE)
        y = y.reshape(-1).to(DEVICE)
        optimizer.zero_grad()
        outputs = model(inp)

        loss = F.cross_entropy(outputs, y)
        

        loss.backward()
        epoch_train_loss += loss.detach().cpu()

        optimizer.step()
        scheduler.step()
    
    # epoch_train_loss = epoch_train_loss/len(unlabelled_dl)
    # writer.add_scalar('Loss_Epoch', epoch_train_loss, epoch)
    # labelled_loader.reset()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/199 [00:00<?, ?it/s]

## More Pytorch Profiler

In [44]:
model = models.resnet18()
model.fc = nn.Linear(model.fc.in_features, len(labelled_iterator.label_list), True)
model = model.to(DEVICE)

"""
logger = AutoLogger({'precision': lambda gt, pred: (gt.argmax(dim=-1)==pred).mean()}, tensorboard=True)
for epoch in range(2):
    logger.epoch_start()
    # train_part
    for i in range(2):
        y_true = np.random.randint(0,2, size=(5,))
        y_pred = np.random.randint(0,2, size=(5,))
        logger.log_loss(0, 'train')
        logger.log_metrics(y_true, y_pred, 'train')

    # test part
    for i in range(1):
        y_true = np.random.randint(0,2, size=(5,))
        y_pred = np.random.randint(0,2, size=(5,))
        logger.log_loss(1, 'test')
        logger.log_metrics(y_true, y_pred, 'test')

    logger.epoch_end()
"""
# model = torch.nn.DataParallel(ImModel(models.resnet18, 1000, None), device_ids=[0,1,2,3]).cuda()

optimizer = optim.AdamW(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.OneCycleLR(optimizer, max_lr=3e-3, 
                                          epochs=EPOCHS, 
                                          steps_per_epoch=len(pytorch_dl))


with torch.profiler.profile(
        activities=[torch.profiler.ProfilerActivity.CPU, torch.profiler.ProfilerActivity.CUDA],
        schedule=torch.profiler.schedule(wait=1, warmup=1, active=3, repeat=2),
        on_trace_ready=torch.profiler.tensorboard_trace_handler('./profiler/dali-expanded'),
        record_shapes=True,
        profile_memory=True,
        with_stack=True
) as prof:
    for epoch in tqdm(range(1), leave=True):
        
        epoch_train_loss = 0
        epoch_test_loss = 0
        
        for i, (labelled_data) in enumerate(tqdm(labelled_loader, total=len(pytorch_dl), leave=False)):
            # lab, unlab = batch
            inp, y = labelled_data[0]['x'], labelled_data[0]['y']
            inp = inp.to(DEVICE)
            y = y.reshape(-1).to(DEVICE)
            optimizer.zero_grad()
            outputs = model(inp)

            loss = F.cross_entropy(outputs, y)
            

            loss.backward()
            epoch_train_loss += loss.detach().cpu()

            optimizer.step()
            scheduler.step()
            prof.step()
        
        # epoch_train_loss = epoch_train_loss/len(unlabelled_dl)
        # writer.add_scalar('Loss_Epoch', epoch_train_loss, epoch)
        labelled_loader.reset()

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/199 [00:00<?, ?it/s]

STAGE:2023-08-13 17:42:41 3306472:3306472 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-08-13 17:42:42 3306472:3306472 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-08-13 17:42:42 3306472:3306472 ActivityProfilerController.cpp:321] Completed Stage: Post Processing
STAGE:2023-08-13 17:42:42 3306472:3306472 ActivityProfilerController.cpp:311] Completed Stage: Warm Up
STAGE:2023-08-13 17:42:42 3306472:3306472 ActivityProfilerController.cpp:317] Completed Stage: Collection
STAGE:2023-08-13 17:42:42 3306472:3306472 ActivityProfilerController.cpp:321] Completed Stage: Post Processing


In [45]:
print(prof.key_averages(group_by_input_shape=True).table(sort_by="cuda_time_total", row_limit=10))

-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  --------------------------------------------------------------------------------  
                                                   Name    Self CPU %      Self CPU   CPU total %     CPU total  CPU time avg     Self CUDA   Self CUDA %    CUDA total  CUDA time avg       CPU Mem  Self CPU Mem      CUDA Mem  Self CUDA Mem    # of Calls                                                                      Input Shapes  
-------------------------------------------------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  ------------  -------------------------------------------------------------------------